In [52]:
#INSTRUCTIONS:
# Hi!  Run me first (hit the play button above)


import pandas as pd
import numpy as np
import csv
#matplotlib
%pylab

from pandas import DataFrame, Series

# precision is a handy (albeit somewhat hidden) way to tell Pandas to only
# display two decimal places of precision in the numbers we generate in our dataframes
pd.set_option('precision',2)

# Read in the .csv.
# note: I will provide the eecm.csv spreadsheet, but it has been modified.
# I used OpenRefine to clean it up a bit.  I eliminated dollar signs from gift amounts.
# I also tweaked a couple of zip codes which were obviously in error.  These changes will not 
# appreciably affect the data or metrics we derive.
# Also noteworthy: there were a substantial number of "unknown" zip codes.
eecm=pd.read_csv('eecm.csv',index_col='Constituent ID')

# Now, a bit more data cleaning:
# this loop runs through the data for all available years (2008 to 2018)
# and formats a few key columns of it as numeric values, to enable us to process it.
for i in range(8,18):
    if i<10:
        startYear='0'+str(i)
        if i == 8:
            endYear='0'+str(i+1)
        if i == 9:
            endYear=str(i+1)
    else:
        startYear=str(i)
        endYear=str(i+1)

    eecm['Gift Amount '+startYear+'-'+endYear]=pd.to_numeric(eecm['Gift Amount '+startYear+'-'+endYear],errors='coerce',downcast='float')
    eecm['Appeals in '+startYear+'-'+endYear]=pd.to_numeric(eecm['Appeals in '+startYear+'-'+endYear],errors='coerce',downcast='float')
    eecm['Gift Count '+startYear+'-'+endYear]=pd.to_numeric(eecm['Gift Count '+startYear+'-'+endYear],errors='coerce',downcast='float')

# Here, I initially thought to add columns for each year, and grow by concatenating the df object
# but, this is probably better done on a per-year basis, using Jupyter notebooks and user queries.


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [53]:
# INSTRUCTIONS:
# run me second.  This cell fetches gift and appeal data for a year of your choosing


#------------------------------------------------------#
# code follows:

# this prompts the user to input a desired year for which to fetch gift and appeal stats
desiredYear=input("What year would you like to query?  e.g 09-10: ")

# this fetches a dataframe from our spreadsheet, with totals for Appeals, Gift Count, and Gift Amount
# grouped by zip code
test2=eecm.groupby(['Zip Code']).agg({'Appeals in '+desiredYear:sum,'Gift Count '+desiredYear:sum,'Gift Amount '+desiredYear:sum})

# LET'S ADD SOME CALCULATED COLUMNS:

#first: 'Response Rate' to appeals.  That is, the number of gifts per appeal
# ** note: this may show up as 'inf' if no appeals were made (which is the case in several years)
test2[desiredYear+' Response Rate']=test2['Gift Count '+desiredYear]/test2['Appeals in '+desiredYear]

# second: 'Avg Gift' per zip
test2['Avg Gift '+desiredYear]=test2['Gift Amount '+desiredYear]/test2['Gift Count '+desiredYear]

#third: '$/Appeal' or the dollar amount generated per appeal -- note: the amount includes dollars
# that weren't necessarily generated in response to an appeal.  That is, gifts were made from donors
# to whom no appeal was made.
test2['$/Appeal '+desiredYear]=test2['Gift Amount '+desiredYear]/test2['Appeals in '+desiredYear]

# for display purposes, let's just look at zips with significant total donations
# say... $1000 or more (in total)
# this could easily be made variable.  However, I have found looking at zip codes with low total 
# gift amounts to be counterproductive to getting a feel for the benchmarks
test3=test2.loc[test2['Gift Amount '+desiredYear]>1000]

# here's our output, sorted by the total number of appeals for each zip, in the desired year:
test3.sort_values(by=['Appeals in '+desiredYear])



,Appeals in 08-09,Gift Count 08-09,Gift Amount 08-09,08-09 Response Rate,Avg Gift 08-09,$/Appeal 08-09
Zip Code,,,,,,
15229,18.0,13.0,1005.00,0.72,77.31,55.83
15136,34.0,14.0,1339.00,0.41,95.64,39.38
15202,39.0,18.0,1456.00,0.46,80.89,37.33
16066,41.0,13.0,1790.00,0.32,137.69,43.66
15216,42.0,9.0,1106.00,0.21,122.89,26.33
15241,45.0,9.0,1120.00,0.20,124.44,24.89
15224,46.0,23.0,1409.00,0.50,61.26,30.63
15228,53.0,12.0,1080.00,0.23,90.00,20.38
15222,53.0,14.0,2724.00,0.26,194.57,51.40


In [54]:
#INSTRUCTIONS:

# run me third.  This cell gives a 'feel' for the data we've been pulling.
# Of particular interest may be the min, max, mean, and std (standard deviation)
# of one's response rate.  Response rates in the output cell (above) that are
# more than two standard deviations away from the mean are approaching outlier status

test3.describe()

,Appeals in 08-09,Gift Count 08-09,Gift Amount 08-09,08-09 Response Rate,Avg Gift 08-09,$/Appeal 08-09
count,25.00,25.00,25.00,25.00,25.00,25.00
mean,175.36,52.00,3820.90,0.33,84.16,26.95
std,189.68,56.06,3812.16,0.13,31.66,12.04
min,18.00,9.00,1005.00,0.12,43.11,9.46
25%,46.00,14.00,1409.00,0.24,67.13,19.62
50%,87.00,30.00,2099.60,0.31,78.14,24.13
75%,207.00,69.00,4651.00,0.40,84.08,33.44
max,717.00,223.00,14724.74,0.72,194.57,55.83


In [50]:
#test3.reset_index(level=0, inplace=True)
# coding note: resetting the index as below is problematic.  If you run this cell twice in a row
# you'll get an error.  The above method avoids that.
#test3 = test3.reset_index()
test3.columns
#boxplot = test3.boxplot(column=['Zip Code', desiredYear+' Response Rate'])

Index(['Appeals in 16-17', 'Gift Count 16-17', 'Gift Amount 16-17',
       '16-17 Response Rate', 'Avg Gift 16-17', '$/Appeal 16-17'],
      dtype='object')

In [62]:
#INSTRUCTIONS:
# run this cell to see a bar plot of the response rate by zip code

fig = test3[desiredYear+' Response Rate'].plot.bar()
##ax=fig.add_subplot()
# I would like to add-in dynamically-drawn sets of standard deviation lines, for visual guidance
#ax.axhline(.08)
